## ひろしまQuest2020#stayhome【コース予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_11.f'
SUBMIT = 'submit/course_1.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 158)

### 不要な列を削除

In [4]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [5]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [6]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 143)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,batter_cnt_bat,bat_game_cnt_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,foreigner,batter_cnt,bat_game_cnt,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.477023,0.097592,0.200870,0.074454,0.046799,0.041882,0.008619,0.052760,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.459832,0.090366,0.161893,0.068313,0.099045,0.061079,0.012505,0.046967,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.469730,0.066062,0.159395,0.031361,0.162940,0.062722,0.014112,0.033679,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.477023,0.097592,0.200870,0.074454,0.046799,0.041882,0.008619,0.052760,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.459832,0.090366,0.161893,0.068313,0.099045,0.061079,0.012505,0.046967,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 143)

### test
- 行数: 521650

In [8]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 143)

In [9]:
del all_pitch
gc.collect()

20

In [10]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [11]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [12]:
lgb_train = lgb.Dataset(train_d, train['pitch_area'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [13]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
        'seed' : 0,
        'learning_rate' : 0.1,
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 2.34479 + 0.000779427
[200]	cv_agg's multi_logloss: 2.34551 + 0.00114307
Best num_boost_round: 122
Best CV score: 2.3445089434874253
CPU times: user 44min 20s, sys: 38 s, total: 44min 58s
Wall time: 12min 2s


In [14]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [15]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 9min 35s, sys: 3.55 s, total: 9min 39s
Wall time: 2min 28s


### Feature Importance

In [16]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
136,age_dif_b-c,613
6,player_cnt_in_game,616
138,salary_x_year_dif_b-c,637
76,pit_game_cnt_pit,639
122,salary_dif_p-b,641
134,salary_dif_b-c,671
97,bat_game_cnt_bat,677
72,changeup_pit,678
64,height_pit,683
19,bc_shoot,694


In [17]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 2min 58s, sys: 1.83 s, total: 3min
Wall time: 48.1 s


## 結果出力

In [18]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.064006,0.143549,0.064409,0.045495,0.074420,0.036198,0.045588,0.043095,0.032021,0.182670,0.072917,0.103635,0.091997
1,1,0.074659,0.141996,0.060242,0.039799,0.054641,0.048694,0.035802,0.084526,0.030848,0.167755,0.057960,0.113144,0.089935
2,2,0.035430,0.079890,0.047384,0.031454,0.038954,0.040059,0.036782,0.049679,0.028300,0.198056,0.098810,0.190338,0.124865
3,3,0.046948,0.122164,0.050405,0.027868,0.054973,0.035580,0.033341,0.047457,0.025697,0.198255,0.100684,0.137062,0.119566
4,4,0.050024,0.166118,0.054775,0.034160,0.059670,0.045845,0.037967,0.060731,0.028789,0.158934,0.070328,0.118202,0.114456


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [19]:
submit.mean()

index    260824.500000
0             0.048000
1             0.068148
2             0.056640
3             0.046695
4             0.060934
5             0.047570
6             0.043053
7             0.065031
8             0.054208
9             0.109530
10            0.087978
11            0.133396
12            0.178817
dtype: float64

In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [21]:
'signate submit --competition-id=276 ~/work/baseball/{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=276 ~/work/baseball/submit/course_1.csv --note feat=143_cv=2.3445089434874253'

### GCS

In [22]:
'gsutil cp ~/work/baseball/{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ~/work/baseball/submit/course_1.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648